In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms
import os

import time 

In [2]:
# Определяем трансформации для изображений
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Все изображения приведём к размеру 32x32
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Пути к твоим данным с пандами
data_dir = './data'  # Замени на путь к папке data
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Загружаем данные с красными пандами для каждого набора
train_panda_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_panda_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_panda_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Загружаем CIFAR-10
cifar10_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Класс для метки "не панда" (1) для CIFAR-10
class CIFAR10AsNonPanda(torch.utils.data.Dataset):
    def __init__(self, cifar_dataset):
        self.cifar_dataset = cifar_dataset

    def __len__(self):
        return len(self.cifar_dataset)

    def __getitem__(self, idx):
        image, _ = self.cifar_dataset[idx]  # CIFAR-10 имеет свои метки, мы их не используем
        return image, 1  # Все изображения CIFAR-10 метим как "не панда" (1)

# Оборачиваем CIFAR-10 в наш новый класс для тренировочного и тестового наборов
non_panda_train_dataset = CIFAR10AsNonPanda(cifar10_train_dataset)
non_panda_test_dataset = CIFAR10AsNonPanda(cifar10_test_dataset)

# Объединяем panda и non-panda датасеты для каждого набора
train_dataset = ConcatDataset([train_panda_dataset, non_panda_train_dataset])
val_dataset = ConcatDataset([val_panda_dataset, non_panda_test_dataset])
test_dataset = ConcatDataset([test_panda_dataset, non_panda_test_dataset])

# Создаем DataLoader для каждого набора
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Определим архитектуру нейросети
class PandaClassifier(nn.Module):
    def __init__(self):
        super(PandaClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 класса: "панда" (0) и "не панда" (1)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 32 * 8 * 8)  # Разворачиваем тензор перед полносвязным слоем
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Инициализация модели, функции потерь и оптимизатора
model = PandaClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

st_time = time.time() 
print("Время пошло")
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {accuracy:.2f}%')

# Тестирование модели
def test_model(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

# Тестируем модель
test_model(test_loader, model)

end_time = time.time()  # Засекаем время после выполнения

elapsed_time = end_time - st_time
print(f"Время выполнения: {elapsed_time} секунд")

Files already downloaded and verified
Files already downloaded and verified
Время пошло
Epoch [1/10], Loss: 0.0621
Validation Loss: 0.0887, Accuracy: 96.67%
Epoch [2/10], Loss: 0.0398
Validation Loss: 0.0734, Accuracy: 97.63%
Epoch [3/10], Loss: 0.0313
Validation Loss: 0.0524, Accuracy: 98.42%
Epoch [4/10], Loss: 0.0249
Validation Loss: 0.0378, Accuracy: 98.86%
Epoch [5/10], Loss: 0.0201
Validation Loss: 0.0396, Accuracy: 98.78%
Epoch [6/10], Loss: 0.0169
Validation Loss: 0.0340, Accuracy: 98.91%
Epoch [7/10], Loss: 0.0136
Validation Loss: 0.0375, Accuracy: 98.93%
Epoch [8/10], Loss: 0.0110
Validation Loss: 0.0390, Accuracy: 99.00%
Epoch [9/10], Loss: 0.0088
Validation Loss: 0.0544, Accuracy: 98.67%
Epoch [10/10], Loss: 0.0073
Validation Loss: 0.0751, Accuracy: 98.45%
Accuracy on test set: 99.45%
Время выполнения: 256.5696861743927 секунд


In [3]:
torch.save(model.state_dict(), 'CPU_BigData_panda_classifier_state_dict.pth')